# Updating Graphs on Interactions - Part 3

We are going to add another layout to show some markdown text, and make the acceleration line thickness proportional to the number of cylinders.

In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.graph_objs as go

import pandas as pd
from numpy import random

In [2]:
app = dash.Dash()

In [3]:
df = pd.read_csv('mpg.csv')

In order to add a bit of noise to the data (year column):

In [4]:
df['year'] = random.randint(-4,5,len(df))*0.1 + df['model_year'] # for example: 1978 → 1977.7

In [5]:
app.layout = html.Div([html.Div([dcc.Graph(id='mpg-scatter',
                                           figure={'data':[go.Scatter(x=df['year']+1900,
                                                                      y=df['mpg'],
                                                                      text = df['name'],
                                                                      hoverinfo = ['text','y','x'],
                                                                      mode = 'markers')
                                                          ],
                                                   'layout' : go.Layout(title='MPG Data',
                                                                        xaxis={'title':'Model Year'},
                                                                        yaxis={'title':'MPG'},
                                                                        hovermode='closest')
                                                  }),
                                ], style={'width':'50%','display':'inline-block'}),
                       html.Div([dcc.Graph(id='mpg_line',
                                           figure={'data':[go.Scatter(x=[0,1],
                                                                      y=[0,1],
                                                                      mode='lines')],
                                                   'layout':go.Layout(title='Acceleration', margin={'l':0})}
                                          )],
                                style={'width':'20%','height':'50%','display':'inline-block'}),
                       html.Div([dcc.Markdown(id='mpg_stats')],
                                style={'width':'20%',
                                       'height':'50%',
                                       'display':'inline-block'})
                      ])

In [6]:
@app.callback(Output('mpg_line','figure'),
              [Input('mpg-scatter','hoverData')])
def callback_graph(hoverData):
    v_index = hoverData['points'][0]['pointIndex'] #get vehicule index from json to select from the df
    figure = {'data':[go.Scatter(x=[0,1],
                                 y=[0,60/df.iloc[v_index]['acceleration']],
                                 mode='lines',
                                line = {'width':2*df.iloc[v_index]['cylinders']})], #we normalize the accelerations, how many secs in get from 0-60 mph
              'layout':go.Layout(title=df.iloc[v_index]['name'],
                                 xaxis={'visible':False},
                                 yaxis={'visible':False, 'range':[0,60/df['acceleration'].min()]}, # in order to keep the y axis constant, and see the line moving
                                 margin={'l':0}, #we must keep sending back these options not to get errors when refreshing
                                 height=300)} 
    return figure #important to be able to refresh!!


Now we create another function and callback to show the text:

In [9]:
@app.callback(Output('mpg_stats','children'),
              [Input('mpg-scatter','hoverData')])
def callback_stats(hoverData):
    v_index = hoverData['points'][0]['pointIndex'] #get vehicule index from json to select from the df
    stats="""        
             {} cylinders
             {}cc displacement
             0 to 60mph in {} seconds
          """.format(df.iloc[v_index]['cylinders'],
                     df.iloc[v_index]['displacement'],
                     df.iloc[v_index]['acceleration'])
    return stats

In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Nov/2020 02:18:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:48] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:48] "GET /_dash-component-suites/dash_core_components/async-markdown.v1_13_0m1604001894.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\

127.0.0.1 - - [23/Nov/2020 02:18:48] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Alejandro\anaconda3\envs\Udemy\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\

127.0.0.1 - - [23/Nov/2020 02:18:48] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [23/Nov/2020 02:18:48] "GET /_dash-component-suites/dash_core_components/async-highlight.v1_13_0m1604001894.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02

127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:18:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [23/Nov/2020 02:19:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2020 02:19:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.